# Sampling Validation using MC

We can also validate our SBI model by drawing samples from the posterior using MCMC methods. This allows us to assess how well our model captures the underlying distribution of parameters given the observed data.

We can do this using the ```SBI_Fitter.fit_observation_using_sampler``` method, which lets us choose from a few nested sampling or MCMC samplers. Here, we'll use the `dynesty` sampler to draw samples from the posterior.

First we'll load a trained model and choose a mock observation to fit.

In [1]:
from synference import SBI_Fitter, load_unc_model_from_hdf5, test_data_dir

library_path = (
    f"{test_data_dir}/grid_BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_2.7_Calzetti_v3_multinode.hdf5"  # noqa: E501
)

fitter = SBI_Fitter.load_saved_model(
    model_file=f"{test_data_dir}", library_path=library_path, device="cpu"
)

nm_path = f"{test_data_dir}/BPASS_DenseBasis_v4_final_nsf_0_params_empirical_noise_models.h5"
noise_models = load_unc_model_from_hdf5(nm_path)

fitter.feature_array_flags["empirical_noise_models"] = noise_models

ValueError: Model file /Users/user/Library/Application Support/Synthesizer/data/synference does not exist.

First we can recreate the simulator from the information stored in the library. We will use the simulator in the log-likelihood calculation during sampling.


In [ ]:
fitter.recreate_simulator_from_library(
    override_library_path=library_path, override_grid_path="test_grid.hdf5"
);

Then we can proceed to fit the observation using the sampler.

The code will attempt to remove parameters which don't affect the fit (e.g. supplemental parameters) before fitting, but you can also specify to remove specific parameters using the `remove_params` argument.

Let's choose an observation from the validation set and fit it using the dynesty sampler.


In [ ]:
index = 2

data = f"{test_data_dir}/sbi_test_data_BPASS_DenseBasis_v4_final.npz"
import numpy as np

loaded = np.load(data)
X_test = loaded["X"][:, index]
y_test = loaded["y"][:, index]

print(X_test)

We'll just recreate our prior for the model.

In [ ]:
fitter.create_priors(set_self=True);

No we can run the sampler to fit the observation. This is too computationally expensive to run here, so we'll just show the code you would use.

```python
result = fitter.fit_observation_using_sampler(
    observation=X_test,
    sampler="dynesty",
    sampler_kwargs={"bound": "multi", "sample": "rwalk", "run_kwargs": {"n_effective": 2000}},
    plot_name=f"example_{index}",
    min_flux_pc_error=0.05,
)
samples = result["samples"]
log_l = result["logl"]
log_w = result["logwt"]

```

We can sample using our trained SBI model as well for comparison.

In [ ]:
predicted_params = fitter.sample_posterior(X_test, num_samples=1000)